<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/mobilenetV2_TL_and_cash_09012018_90_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [0]:
import sys
sys.path.append('/content/models/research/slim')

In [0]:
import os
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
print( os.listdir('/content') )

Saving imagenet_class_names.txt to imagenet_class_names (2).txt
User uploaded file "imagenet_class_names.txt" with length 31674 bytes
['.config', 'imagenet_class_names (2).txt', 'imagenet_class_names.txt', 'my_mnist_model_5_to_9_five_frozen.index', 'models', 'my_mnist_model_5_to_9_five_frozen.meta', 'flower_photos', 'checkpoint', 'my_mnist_model_5_to_9_five_frozen.data-00000-of-00001', 'imagenet_class_names (1).txt', 'mobilenet_v2_1', 'sample_data']


In [0]:


#Загрузите последнюю предварительную модель Inception v3: контрольная точка доступна по адресу https://github.com/tensorflow/models/tree/master/research/slim . 
#писок имен классов доступен по адресу https://goo.gl/brXRtZ , но вы должны сначала вставить «background» класс.
import os
import re 
import sys
import tarfile
import numpy as np
import tensorflow as tf
from random import sample
from six.moves import urllib
import matplotlib.image as mpimg
import tensorflow.contrib.slim as slim
#from tensorflow.contrib.slim.nets import inception
from nets.mobilenet import mobilenet_v2
from skimage.transform import rescale, resize, downscale_local_mean

MODEL_NAME = "mobilenet_v2_1.0_224.ckpt"
CLASS_NAME_REGEX = re.compile(r"^n\d+\s+(.*)\s*$", re.M | re.U)
DATASET_URL = "http://download.tensorflow.org/example_images/flower_photos.tgz"
MODEL_URL = "https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.0_224.tgz"



def download_progress(count, block_size, total_size):
    percent = count * block_size * 100 // total_size
    sys.stdout.write("\rDownloading: {}%".format(percent))
    sys.stdout.flush()
    

def download_tgz(url):
    file_name=os.path.basename(url)
    name_parts = os.path.split(file_name)
    _, name = name_parts
    name_path = name.rsplit( ".", 2 )[ 0 ]
    path = os.path.join("/content", name_path)
    tgz_path = os.path.join(path, name)
    if os.path.exists(path): #and os.path.isfile(tgz_path):
      print("file and path exist ", tgz_path, "Path is:", os.listdir(path), "!")
      return path
    else:
      print(os.path.exists(path), os.path.isfile(tgz_path), path, tgz_path)
      os.makedirs(path, exist_ok=True)
      urllib.request.urlretrieve(url, tgz_path, reporthook=download_progress)
      tmp_tgz = tarfile.open(tgz_path)
      tmp_tgz.extractall(path=path)
      tmp_tgz.close()
      os.remove(tgz_path)
      print("path created:", path)
      return path
    
# Загружаем модель
MODELS_PATH = download_tgz(MODEL_URL)
# Загружаем датасет
FLOWERS_PATH = download_tgz(DATASET_URL)
 

def load_class_names():
    with open(os.path.join("/content", "imagenet_class_names.txt"), "rb") as f:
        content = f.read().decode("utf-8")
        #sys.stdout.flush()
        return CLASS_NAME_REGEX.findall(content)
      
load_class_names()    
class_names = ["background"] + load_class_names()       


file and path exist  /content/mobilenet_v2_1/mobilenet_v2_1.0_224.tgz Path is: ['mobilenet_v2_1.0_224.ckpt.data-00000-of-00001', 'mobilenet_v2_1.0_224_eval.pbtxt', 'mobilenet_v2_1.0_224_frozen.pb', 'mobilenet_v2_1.0_224.ckpt.meta', 'mobilenet_v2_1.0_224.ckpt.index', 'mobilenet_v2_1.0_224.tflite', 'mobilenet_v2_1.0_224_info.txt'] !
file and path exist  /content/flower_photos/flower_photos.tgz Path is: ['flower_photos'] !


In [0]:

"""Convolution blocks for mobilenet."""
import contextlib
import functools

import tensorflow as tf

slim = tf.contrib.slim

# tf.contrib.slim.conv2d
# tf.contrib.slim.separable_conv2d


def _make_divisible(v, divisor, min_value=None):
  if min_value is None:
    min_value = divisor
  new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
  # Make sure that round down does not go down by more than 10%.
  if new_v < 0.9 * v:
    new_v += divisor
  return new_v


def _split_divisible(num, num_ways, divisible_by=8):
  """Равномерно разбивает num, num_ways, поэтому каждый фрагмент кратен divisible_by."""
  assert num % divisible_by == 0
  assert num / num_ways >= divisible_by
  # Примечание: хотите округлить, мы корректируем каждый сплит в соответствии с итогом.
  base = num // num_ways // divisible_by * divisible_by
  result = []
  accumulated = 0
  for i in range(num_ways):
    r = base
    while accumulated + r < num * (i + 1) / num_ways:
      r += divisible_by
    result.append(r)
    accumulated += r
  assert accumulated == num
  return result







def expand_input_by_factor(n, divisible_by=8):
  return lambda num_inputs, **_: _make_divisible(num_inputs * n, divisible_by)

#################################################################################
#[None, 7, 7, 160]
#################################################################################
#@slim.add_arg_scope
def expanded_conv(input_tensor,
                  num_outputs,
                 # expansion_size=expand_input_by_factor(6),
                 # stride=1,
                 # rate=1,
                 # kernel_size=(3, 3),
                 # residual=True,
                 # normalizer_fn=None,    # normalizer_fn
                 # project_activation_fn=tf.identity,
                 # split_projection=1,
                 # split_expansion=1,
                 # expansion_transform=None,
                  depthwise_location,
                 # depthwise_channel_multiplier=1,
                 # endpoints=None,
                 # use_explicit_padding=False,
                 # padding='SAME',
                 # scope=None
    
                         ):
    #depthwise_location='expansion',
    expansion_size=expand_input_by_factor(6)
 # with tf.variable_scope(scope, default_name='expanded_conv') as s, \
 #      tf.name_scope(s.original_name_scope):
    prev_depth = input_tensor.get_shape().as_list()[3]  ########################  prev_depth = 160
 #   if  depthwise_location not in [None, 'input', 'output', 'expansion']:
 #     raise TypeError('%r is unknown value for depthwise_location' %
 #                     depthwise_location)
 #   if use_explicit_padding:                            #######################  use_explicit_padding = False
 #     if padding != 'SAME':
 #       raise TypeError('`use_explicit_padding` should only be used with '
 #                       '"SAME" padding.')
 #     padding = 'VALID'
    depthwise_func = functools.partial(
        slim.separable_conv2d,
        num_outputs=None,
        kernel_size=(3, 3),
        depth_multiplier=1, #depthwise_channel_multiplier
        stride=1,
        rate=1,
        normalizer_fn=slim.batch_norm,
        padding='SAME',
        scope='depthwise')
    # b1 -> b2 * r -> b2
    #   i -> (o * r) (bottleneck) -> o
    
    input_tensor = tf.identity(input_tensor, 'input')
    net = input_tensor

    if depthwise_location == 'input':
  #    if use_explicit_padding:                          #######################  use_explicit_padding = False
  #      net = _fixed_padding(net, kernel_size, rate)
      net = depthwise_func(net, activation_fn=None)
      print("net = depthwise_func(net, activation_fn=None, )", "net =", net  )

    if callable(expansion_size):
      inner_size = expansion_size(num_inputs=prev_depth)
      print("callable(expansion_size)", callable(expansion_size),"inner_size", inner_size)
      ########################  prev_depth = 160 inner_size = 960
    else:
      inner_size = expansion_size
      print("callable(expansion_size)", callable(expansion_size),"inner_size", inner_size)
    if inner_size > net.shape[3]:                        #######################   [None, 7, 7, 160]
      print("if inner_size > net.shape[3]", "inner_size=", inner_size, "net.shape[3] =", net.shape[3])
      net = split_conv(
          net,
          inner_size,
          num_ways= 1, #split_expansion,
          scope='expand',
          stride=1,
          normalizer_fn=slim.batch_norm)
#      print("net = slim.conv2d(net(", net.get_shape().as_list(), "),inner_size =", inne_size, "num_ways= 1, #split_expansion,scope='expand', stride=1,normalizer_fn=slim.batch_norm))", "inner_size", inner_size, "net=", net)
      net = tf.identity(net, 'expansion_output')
#    if endpoints is not None:
#      endpoints['expansion_output'] = net

    if depthwise_location == 'expansion':
#      if use_explicit_padding:                          #######################  use_explicit_padding = False
#        net = _fixed_padding(net, kernel_size, rate)
      net = depthwise_func(net)
      print("net = slim.separable_conv2d(net", net.get_shape().as_list(), ", num_outputs=None,kernel_size=(3, 3), depth_multiplier=1, stride=1, rate=1, normalizer_fn=slim.batch_norm, padding='SAME',scope='depthwise')")
     
    net = tf.identity(net, name='depthwise_output')
   # if endpoints is not None:
   #   endpoints['depthwise_output'] = net
   # if expansion_transform:                             #######################  expansion_transform = None
   #   net = expansion_transform(expansion_tensor=net, input_tensor=input_tensor)
    #Обратите внимание, что в отличие от расширения, у нас всегда есть проекция для получения желаемого выходного размера.
    net = split_conv(
        net,
        num_outputs,
        num_ways=1,
        stride=1,
        scope='project',
        normalizer_fn=slim.batch_norm,
        activation_fn=tf.identity)
#    print("net = slim.conv2d(net(", net.get_shape().as_list(), "), num_outputs =", num_outputs=", "num_ways= 1, #split_expansion,scope='project', stride=1, normalizer_fn=slim.batch_norm, activation_fn=tf.identity)")
         
#    if endpoints is not None:
#      endpoints['projection_output'] = net
    if depthwise_location == 'output':
#      if use_explicit_padding:                          #######################  use_explicit_padding = False
#        net = _fixed_padding(net, kernel_size, rate)
      net = depthwise_func(net, activation_fn=None)
      print("net = slim.separable_conv2d(net(", net.get_shape().as_list(), "), num_outputs=None,kernel_size=(3, 3), depth_multiplier=1, depthwise_channel_multiplier, stride=1, rate=1, normalizer_fn=slim.batch_norm, padding='SAME',scope='depthwise', activation_fn=None)")
 #   if callable(residual):  # custom residual/ таможенный остаток
    #net = residual(input_tensor=input_tensor, output_tensor=net)
  
  
  
  
    if (net.get_shape().as_list()[3] == input_tensor.get_shape().as_list()[3]):
        net += input_tensor
        print("net += input_tensor")
      
      
    if (net.get_shape().as_list()[3] == input_tensor.get_shape().as_list()[3]):
#          print("if (net.get_shape().as_list()[3] == input_tensor.get_shape().as_list()[3]):", net.get_shape().as_list()[3], "==", input_tensor.get_shape().as_list()[3])
          net += input_tensor
          print("net += input_tensor")
    return tf.identity(net, name='output')


def split_conv(input_tensor,
               num_outputs,
               num_ways,
               scope,
               divisible_by=8,
               **kwargs):

  b = input_tensor.get_shape().as_list()[3]

  if num_ways == 1 or min(b // num_ways,
                          num_outputs // num_ways) < divisible_by:
    # Don't do any splitting if we end up with less than 8 filters
    # on either side.
    print("net = slim.conv2d(net", input_tensor.get_shape().as_list(), ", num_outputs=", num_outputs, ", [1, 1], scope=", scope, "**kwargs)")
    return slim.conv2d(input_tensor, num_outputs, [1, 1], scope=scope, **kwargs)
    

  outs = []
  print("outs = []")
  input_splits = _split_divisible(b, num_ways, divisible_by=divisible_by)
  print("input_splits =", input_splits)
  output_splits = _split_divisible(
      num_outputs, num_ways, divisible_by=divisible_by)
  print("output_splits =", output_splits)
  inputs = tf.split(input_tensor, input_splits, axis=3, name='split_' + scope)
  print("inputs = tf.split(input_tensor, input_splits, axis=3, name='split_' + scope)")
  base = scope
  for i, (input_tensor, out_size) in enumerate(zip(inputs, output_splits)):
    scope = base + '_part_%d' % (i,)
    n = slim.conv2d(input_tensor, out_size, [1, 1], scope=scope, **kwargs)
    n = tf.identity(n, scope + '_output')
    outs.append(n)
  print("net = tf.concat(outs=", outs, ", 3, name=", scope, "+ '_concat')")  
  return tf.concat(outs, 3, name=scope + '_concat')



#net = slim.conv2d(net [None, 7, 7, 160] , num_outputs= 960 , [1, 1], scope= 'expand', stride=1, normalizer_fn=slim.batch_norm)

#net = slim.separable_conv2d(net [None, 7, 7, 960] , num_outputs=None, kernel_size=(3, 3), depth_multiplier=1, stride=1, rate=1, normalizer_fn=slim.batch_norm, padding='SAME',scope='depthwise')

#net = slim.conv2d(net [None, 7, 7, 960] , num_outputs= 320 , [1, 1], scope= 'project', normalizer_fn=slim.batch_norm, activation_fn=tf.identity)
#net = slim.conv2d(net,  stride=1, kernel_size=[1, 1], num_outputs=1280)
#net = tf.nn.avg_pool( net [None, 7, 7, 160] ksize= [1, 7, 7, 1] strides=[1, 1, 1, 1], padding='VALID')




In [0]:
def global_pool(input_tensor):

  pool_op=tf.nn.avg_pool
  shape = input_tensor.get_shape().as_list()
  if shape[1] is None or shape[2] is None:
    print("if shape[1] is None or shape[2] is None:", "shape[1]=". shape[1], "shape[2]=", shape[2])
    kernel_size = tf.convert_to_tensor(
        [1, tf.shape(input_tensor)[1],
         tf.shape(input_tensor)[2], 1])
  else:
    
    kernel_size = [1, shape[1], shape[2], 1]
    
  output = pool_op(
      input_tensor, ksize=kernel_size, strides=[1, 1, 1, 1], padding='VALID')
  print("net = tf.nn.avg_pool( net", input_tensor.get_shape().as_list(), "ksize=", kernel_size, "strides=[1, 1, 1, 1], padding='VALID')")
  # Recover output shape, for unknown shape.
  output.set_shape([None, 1, 1, None])
  #print("net = output.set_shape([None, 1, 1, None])", output.get_shape().as_list())
  return output

In [0]:
width = 224 #299
height = 224 #299
channels = 3
import tensorflow as tf
tf.reset_default_graph()
n_epochs = 1000
batch_size = 100
best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 





#Каждый подкаталог flower_photos каталога содержит все изображения данного класса. Давайте получим список классов:
flowers_root_path = os.path.join(FLOWERS_PATH, "flower_photos")
flower_classes = sorted([dirname for dirname in os.listdir(flowers_root_path)
                  if os.path.isdir(os.path.join(flowers_root_path, dirname))])

#Давайте получим список всех путей файлов изображений для каждого класса:

from collections import defaultdict
image_paths = defaultdict(list)
for flower_class in flower_classes:
    image_dir = os.path.join(flowers_root_path, flower_class)
    for filepath in os.listdir(image_dir):
        if filepath.endswith(".jpg"):
            image_paths[flower_class].append(os.path.join(image_dir, filepath))
            
            
#Давайте сортируем пути изображения, чтобы заставить этот ноутбук вести себя последовательно на нескольких запусках:
for paths in image_paths.values():
    paths.sort() 

    
#Для получения дополнительных функций манипуляции с изображениями, таких как вращения, проверьте документацию SciPy или эту приятную страницу .
from scipy.misc import imresize
def prepare_image(image, target_width = width, target_height = height, max_zoom = 0.2):
    """Zooms and crops the image randomly for data augmentation."""

    # First, let's find the largest bounding box with the target size ratio that fits within the image
    height = image.shape[0]
    width = image.shape[1]
    image_ratio = width / height
    target_image_ratio = target_width / target_height
    crop_vertically = image_ratio < target_image_ratio
    crop_width = width if crop_vertically else int(height * target_image_ratio)
    crop_height = int(width / target_image_ratio) if crop_vertically else height
        
    # Now let's shrink this bounding box by a random factor (dividing the dimensions by a random number
    # between 1.0 and 1.0 + `max_zoom`.
    resize_factor = np.random.rand() * max_zoom + 1.0
    crop_width = int(crop_width / resize_factor)
    crop_height = int(crop_height / resize_factor)
    
    # Next, we can select a random location on the image for this bounding box.
    x0 = np.random.randint(0, width - crop_width)
    y0 = np.random.randint(0, height - crop_height)
    x1 = x0 + crop_width
    y1 = y0 + crop_height
    
    # Let's crop the image using the random bounding box we built.
    image = image[y0:y1, x0:x1]

    # Let's also flip the image horizontally with 50% probability:
    if np.random.rand() < 0.5:
        image = np.fliplr(image)

    # Now, let's resize the image to the target dimensions.
    image = imresize(image, (target_width, target_height))
    
    # Finally, let's ensure that the colors are represented as
    # 32-bit floats ranging from 0.0 to 1.0 (for now):
    return image.astype(np.float32) / 255



def prepare_batch(flower_paths_and_classes, batch_size):
    batch_paths_and_classes = sample(flower_paths_and_classes, batch_size)
    images = [mpimg.imread(path)[:, :, :channels] for path, labels in batch_paths_and_classes]
    prepared_images = [prepare_image(image) for image in images]
    X_batch = 2 * np.stack(prepared_images) - 1 # Inception expects colors ranging from -1 to 1
    y_batch = np.array([labels for path, labels in batch_paths_and_classes], dtype=np.int32)
    return X_batch, y_batch 
  
  
def prepare_data(flower_paths_and_classes):
    batch_paths_and_classes = flower_paths_and_classes
    images = [mpimg.imread(path)[:, :, :channels] for path, labels in batch_paths_and_classes]
    prepared_images = [prepare_image(image) for image in images]
    X_data = 2 * np.stack(prepared_images) - 1 # Inception expects colors ranging from -1 to 1
    y_data = np.array([labels for path, labels in batch_paths_and_classes], dtype=np.int32)
    return X_data, y_data  
  
  
#раннняя остановка
def get_model_params():
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, tf.get_default_session().run(gvars))}

def restore_model_params(model_params):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: tf.get_default_graph().get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    tf.get_default_session().run(assign_ops, feed_dict=feed_dict)  
  
  #снова получить начальный график v3. На этот раз воспользуемся trainingзаполнителем

flower_class_ids = {flower_class: index for index, flower_class in enumerate(flower_classes)}


#Будет проще перетасовать набор данных, если мы представим его как список пар filepath / class:
flower_paths_and_classes = []
for flower_class, paths in image_paths.items():
    for path in paths:
        flower_paths_and_classes.append((path, flower_class_ids[flower_class]))
        
#Затем давайте перетасовать набор данных и разделим его на обучающий набор и тестовый набор:


np.random.shuffle(flower_paths_and_classes)
a = flower_paths_and_classes
flower_paths_and_classes_train = a[:int(len(a)*0.6)]
flower_paths_and_classes_valid = a[int(len(a)*0.6):int(len(a)*0.8)]
flower_paths_and_classes_test = a[int(len(a)*0.8):]

n_iterations_per_epoch = len(flower_paths_and_classes_train) // batch_size

print("train:", len(flower_paths_and_classes_train) ,"valid:", len(flower_paths_and_classes_valid), "test", len(flower_paths_and_classes_test))

#flower_paths_and_classes
#print("flower_paths_and_classes", len(flower_paths_and_classes))
#print("flower_paths_and_classes_valid", len(flower_paths_and_classes_valid))
X_train, y_train = prepare_data(flower_paths_and_classes_train)
X_test, y_test = prepare_data(flower_paths_and_classes_test)
X_valid, y_valid = prepare_data(flower_paths_and_classes_valid)        
#n_epochs = 10000
#batch_size = 50       

#best_loss_val = np.infty
#check_interval = 100
#checks_since_last_progress = 0
#max_checks_without_progress = 20
#best_model_params = None 

#step

def shuffle_batch(X, batch_size):
  #rnd_idx = [i for i in range(len(X)) if i>=batch_n*batch_size and i<(batch_n*batch_size+batch_size)]
  #print("rnd_idx", rnd_idx)
  
  print("X.shape", X.shape)
  rnd_idx = []
  for i in range(len(X)): 
    rnd_idx.append(i)
  print("rnd_idx.append", rnd_idx)
  for batch_idx in np.array_split(rnd_idx, batch_size):   # np.array_split делит масим на несколько массивов
      #print("batch_idx", batch_idx)  
      X_batch = X[batch_idx]
      
      yield X_batch      


def get_var_list(target_tensor=None):
  
    if target_tensor==None:
        target_tensor = "MobilenetV2/expanded_conv_15/output:0"
    target = target_tensor.split("/")[1]
    all_list = []
    all_var = []

    for var in tf.global_variables():
        if var != []:
            all_list.append(var.name)
            all_var.append(var)
    try:
        all_list = list(map(lambda x:x.split("/")[1],all_list))

        ind = all_list[::-1].index(target)
        ind = len(all_list) -  ind - 1
        print(ind)
        del all_list
        return all_var[:ind+1]
    except:
        print("target_tensor is not exist!")


        
def cash(height, width, channels, flower_classes):
  X = tf.placeholder(tf.float32, shape=[None, height, width, channels], name="X")
  training = tf.placeholder_with_default(False, shape=[])

  with slim.arg_scope(mobilenet_v2.training_scope(is_training=False)): #is_training=False
    logits, end_points = mobilenet_v2.mobilenet(X)

  #with slim.arg_scope(inception.inception_v3_arg_scope()):
      #logits, end_points = inception.inception_v3(X, num_classes=1001, is_training=training)

  inception_saver = tf.train.Saver() 
  n_outputs = len(flower_classes)
  mobilenet_tensor = end_points["layer_17/output"]
  y = tf.placeholder(tf.int32, shape=[None])        
  #target_tensor = "MobilenetV2/expanded_conv_15/output:0"
  #var_list = get_var_list(target_tensor)
  #saver = tf.train.Saver(var_list=var_list)      
  init = tf.global_variables_initializer()      
      
      
      

  ############################## ВЫБИРАЕМ ПЕРЕМЕННЫЕ ##########################

  reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="MobilenetV2") # regular expression
  restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3

  h2_cache_2 = np.empty([0, 7, 7, 160]) 
  print(h2_cache_2)
  h2_cache_valid_2 = np.empty([0, 7, 7, 160])                                                              
  init = tf.global_variables_initializer()
  ################################################################################
  with tf.Session() as sess:
    
      #sess.run(tf.variables_initializer(var_list=train_var))
      inception_saver.restore(sess, os.path.join(MODELS_PATH, MODEL_NAME))
      init.run()
    
     
      for X_batch in shuffle_batch(X_train, batch_size):
           
              h2_cache_1 = sess.run(mobilenet_tensor, feed_dict={X: X_batch, training: False})
          
              h2_cache_2 = np.concatenate((h2_cache_2, h2_cache_1))
           
      for X_val in shuffle_batch(X_valid, batch_size):
              h2_cache_valid_1 = sess.run(mobilenet_tensor, feed_dict={X: X_val, training: False})
            
              h2_cache_valid_2 = np.concatenate((h2_cache_valid_2, h2_cache_valid_1))

  return h2_cache_2, h2_cache_valid_2       
 
    
#h2_cache_2, h2_cache_valid_2 = cash(height, width, channels, flower_classes)
#print(h2_cache_2.shape, h2_cache_valid_2.shape)

train: 2202 valid: 734 test 734


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:73: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [0]:
tf.reset_default_graph()

batch_size = 50
filters=256
kernel_size=3
#inception_saver = tf.train.Saver() 
n_outputs = len(flower_classes)

#y_label = tf.placeholder(tf.int32, (None,10))
ys = tf.placeholder(tf.int32, shape=[None])  
images = tf.placeholder(tf.float32,(None,224,224,3))
with slim.arg_scope(mobilenet_v2.training_scope(is_training=False)): #is_training=False
    logits, end_points = mobilenet_v2.mobilenet(images)
    
with tf.variable_scope("finetune_layers"): 
    
    mobilenet_tensor = tf.get_default_graph().get_tensor_by_name("MobilenetV2/expanded_conv_15/output:0")
    x = slim.conv2d(mobilenet_tensor, num_outputs=960, kernel_size=[1, 1], stride=1)
    x = slim.separable_conv2d(x, num_outputs=None, kernel_size=(3, 3), depth_multiplier=1, stride=1, rate=1,  padding='SAME') #normalizer_fn=slim.batch_norm,
    x = slim.conv2d(x, num_outputs=320, kernel_size=[1, 1], activation_fn=tf.identity)
    x = slim.conv2d(x, stride=1, kernel_size=[1, 1], num_outputs=1280)
    x = tf.nn.avg_pool(x, ksize=[1, 7, 7, 1], strides=[1, 1, 1, 1], padding='VALID')
    dropout = tf.nn.dropout(x, 0.5)
    logits = slim.conv2d(dropout, 10, [1, 1], activation_fn=None, normalizer_fn=None, biases_initializer=tf.zeros_initializer())
    logits = tf.squeeze(logits, [1, 2])
    predictions = slim.softmax(logits)
    
 
'''
####################################layers from mobilnet########################     
    
    x = tf.layers.Conv2D(filters=256,kernel_size=3,name="conv2d_1")(mobilenet_tensor)
    x = tf.nn.relu(x,name="relu_1")
    x = tf.layers.Conv2D(filters=256,kernel_size=3,name="conv2d_2")(x)
    x = tf.layers.Conv2D(10,3,name="conv2d_3")(x)
    logits = tf.reshape(x, (-1,10))
################################################################################
''' 







with tf.name_scope("train1"):     
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=ys)
    loss = tf.reduce_mean(xentropy)
    train_var = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="finetune_layers")
    training_op = optimizer.minimize(loss, var_list=train_var)
    training_op = tf.train.AdamOptimizer().minimize(loss, var_list=train_var)
    #training_op = tf.train.GradientDescentOptimizer(0.0001).minimize(loss, var_list=train_var)   
    
    
    
    
    
with tf.name_scope("eval1"):
    correct = tf.nn.in_top_k(logits, ys, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

#init = tf.global_variables_initializer()
saver = tf.train.Saver() 


'''
def get_var_list(target_tensor=None):
   
    if target_tensor==None:
        target_tensor = "MobilenetV2/expanded_conv_15/output:0"
    target = target_tensor.split("/")[1]
    all_list = []
    all_var = []
    # 遍历所有变量，node.name得到变量名称
    # 不使用tf.trainable_variables()，因为batchnorm的moving_mean/variance不属于可训练变量
    for var in tf.global_variables():
        if var != []:
            all_list.append(var.name)
            all_var.append(var)
    try:
        all_list = list(map(lambda x:x.split("/")[1],all_list))
        # 查找对应变量作用域的索引
        ind = all_list[::-1].index(target)
        ind = len(all_list) -  ind - 1
        print(ind)
        del all_list
        return all_var[:ind+1]
    except:
        print("target_tensor is not exist!")

'''


#target_tensor = "MobilenetV2/expanded_conv_15/output:0"
#var_list = get_var_list(target_tensor)

var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="MobilenetV2")
#saver = tf.train.Saver()
#init = tf.global_variables_initializer()
import time
n_epochs = 100000

from IPython.display import clear_output
max_checks_without_progress = 1000
checks_without_progress = 0
best_loss = np.infty
init = tf.global_variables_initializer() # for Adam
saver = tf.train.Saver(var_list=var_list)  #var_list=var_list)

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    init.run() # for Adam
    sess.run(tf.variables_initializer(var_list=train_var))

    #saver.restore(sess,tf.train.latest_checkpoint(MODELS_PATH))
    saver.restore(sess, os.path.join(MODELS_PATH, MODEL_NAME))
    #(os.path.join(MODELS_PATH, MODEL_NAME)))
    #for var in train_var:
     #   var.initializer.run()

    t0 = time.time()

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(2200)
        for rnd_indices in np.array_split(rnd_idx, 2200 // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={images: X_batch, ys: y_batch})
            #print("h5_batch", h5_batch.shape, "y_batch", y_batch.shape )
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={images: X_valid[:50], ys: y_valid[:50]})
        #loss_val = loss.eval(feed_dict={X: h2_cache_valid_2[:10], ys: y_valid[:10]})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        if epoch % 1 == 0:
          
          
          #clear_output(wait=True)
          print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
              epoch, loss_val, best_loss, acc_val * 100))
          
          #print(tf.train.get_or_create_global_step())
          #print(printm())

    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={images: X_test[:50], ys: y_test[:50]})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))





INFO:tensorflow:Restoring parameters from /content/mobilenet_v2_1/mobilenet_v2_1.0_224.ckpt
0	Validation loss: 0.448671	Best loss: 0.448671	Accuracy: 80.00%
1	Validation loss: 0.440637	Best loss: 0.440637	Accuracy: 86.00%
2	Validation loss: 0.421484	Best loss: 0.421484	Accuracy: 86.00%
3	Validation loss: 0.446935	Best loss: 0.421484	Accuracy: 84.00%
4	Validation loss: 0.479475	Best loss: 0.421484	Accuracy: 88.00%
5	Validation loss: 0.457684	Best loss: 0.421484	Accuracy: 88.00%
6	Validation loss: 0.456539	Best loss: 0.421484	Accuracy: 88.00%
7	Validation loss: 0.608155	Best loss: 0.421484	Accuracy: 84.00%
8	Validation loss: 0.726347	Best loss: 0.421484	Accuracy: 88.00%
9	Validation loss: 0.840778	Best loss: 0.421484	Accuracy: 88.00%
10	Validation loss: 0.779637	Best loss: 0.421484	Accuracy: 88.00%
11	Validation loss: 0.689861	Best loss: 0.421484	Accuracy: 84.00%
12	Validation loss: 0.702121	Best loss: 0.421484	Accuracy: 86.00%


In [0]:
var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="MobilenetV2")
var_list

In [0]:
if target_tensor==None:
        target_tensor = "MobilenetV2/expanded_conv_15/output:0"

target = target_tensor.split("/")[1]
all_list = []
all_var = []
var_plus = []
for var in tf.global_variables():
        if var != []:
            all_list.append(var.name)
            all_var.append(var)
try:
    all_list = list(map(lambda x:x.split("/")[1],all_list))

    #ind = all_list.index(target)
    for i, j in enumerate(all_list):
      if j == target:
          ind = i
          var_plus.append(all_var[i])
          print(var_plus)
    return var_plus[:ind+1]
except:    
    print("target_tensor is not exist!")

In [0]:
train_var

[<tf.Variable 'finetune_layers/conv2d_1/kernel:0' shape=(3, 3, 160, 256) dtype=float32_ref>,
 <tf.Variable 'finetune_layers/conv2d_1/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'finetune_layers/conv2d_2/kernel:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'finetune_layers/conv2d_2/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'finetune_layers/conv2d_3/kernel:0' shape=(3, 3, 256, 10) dtype=float32_ref>,
 <tf.Variable 'finetune_layers/conv2d_3/bias:0' shape=(10,) dtype=float32_ref>]

In [0]:
reset_opt_vars = adam_variables_initializer(optimizer, train_var)
reset_opt_vars

<tf.Operation 'init_1' type=NoOp>

In [0]:
opt = tf.train.AdamOptimizer(name="Adam")
training_op = opt.minimize(loss, var_list=train_var)
print(opt.get_slot_names())
# prints ['momentum']

['m', 'v']


In [0]:
opt.get_slot(some_var, 'Adam')

NameError: ignored

In [0]:
momentum_initializers

[<tf.Operation 'finetune_layers/conv2d_1/kernel/Adam/Assign' type=Assign>,
 <tf.Operation 'finetune_layers/conv2d_1/kernel/Adam_1/Assign' type=Assign>,
 <tf.Operation 'finetune_layers/conv2d_1/bias/Adam/Assign' type=Assign>,
 <tf.Operation 'finetune_layers/conv2d_1/bias/Adam_1/Assign' type=Assign>,
 <tf.Operation 'finetune_layers/conv2d_2/kernel/Adam/Assign' type=Assign>,
 <tf.Operation 'finetune_layers/conv2d_2/kernel/Adam_1/Assign' type=Assign>,
 <tf.Operation 'finetune_layers/conv2d_2/bias/Adam/Assign' type=Assign>,
 <tf.Operation 'finetune_layers/conv2d_2/bias/Adam_1/Assign' type=Assign>,
 <tf.Operation 'finetune_layers/conv2d_3/kernel/Adam/Assign' type=Assign>,
 <tf.Operation 'finetune_layers/conv2d_3/kernel/Adam_1/Assign' type=Assign>,
 <tf.Operation 'finetune_layers/conv2d_3/bias/Adam/Assign' type=Assign>,
 <tf.Operation 'finetune_layers/conv2d_3/bias/Adam_1/Assign' type=Assign>]

In [0]:
saver.restore(sess,tf.train.latest_checkpoint(MODELS_PATH))  

ValueError: ignored